In [12]:
import os
import cv2
from tqdm import tqdm
import json

icdar_path = "E:/Main-Task/icdar17_POD"

dtype = "test"
partitionedfiles_path = os.path.join(icdar_path, "dataset/partitioned")
anno_path = os.path.join(partitionedfiles_path, dtype, "annotations")
img_path = os.path.join(partitionedfiles_path, dtype)
output_annotation_file = partitionedfiles_path + "/POD_" + dtype + ".json"

In [13]:
coco_data = {}
files = os.listdir(anno_path)    #get all files in the folder
cats = []

for i in range(len(files)):
    
    #read the file contents
    fname = anno_path + "/" + files[i]
    f = open(fname, 'r')
    next(f)    #skip the first row
    lines = f.readlines()        
    f.close()   

    #loop to form categories for the list dictionary
    for j in range(1,len(lines)-1,3):  
        start = lines[j].find("<") + 1
        end = lines[j].find("Region")
        cat_name = lines[j][start:end]
        if cat_name not in cats:            
            cats.append(cat_name)
    cats = sorted(cats)        
    
list_dict = []
for i in range(len(cats)):
    tmp = {}
    tmp["supercategory"] = "none"
    tmp["id"] = i
    tmp["name"] = cats[i]
    list_dict.append(tmp)

coco_data["categories"] = list_dict
print(coco_data["categories"])

[{'supercategory': 'none', 'id': 0, 'name': 'figure'}, {'supercategory': 'none', 'id': 1, 'name': 'formula'}, {'supercategory': 'none', 'id': 2, 'name': 'table'}]


In [14]:
coco_data["images"] = []
coco_data["annotations"] = []

img_id = 0
ann_id = 0

for img_id in tqdm(range(len(files))):
    
    #read the file contents
    fname = anno_path + "/" + files[img_id]
    f = open(fname, 'r')
    next(f)    #skip the first row
    lines = f.readlines()        
    f.close()   
    
    
    fname = lines[0][20:28]   #get the name of file without extension
    img_name = fname + ".bmp"
    

    img = img_path + '/' + img_name
    if(not os.path.isfile(img)):
        print('error reading file\n',img)
        continue
    img = cv2.imread(img, 1)
    h, w, c = img.shape

    images_tmp = {}
    images_tmp["file_name"] = img_name
    images_tmp["height"] = h
    images_tmp["width"] = w
    images_tmp["id"] = img_id
    coco_data["images"].append(images_tmp)
    

    j = 1
    while j in range(len(lines)-1):
        
        start = lines[j].find("<") + 1
        end = lines[j].find("Region")
        cat_name = lines[j][start:end]
            
        j += 1
        start = lines[j].find('"') + 1
        end = lines[j].rfind('"')
        coords = lines[j][start:end].split(' ')
        x1, y1 = [int(x) for x in coords[0].split(",")]
        x2, y2 = [int(x) for x in coords[3].split(",")]
        label = cat_name
        

        annotations_tmp = {}
        annotations_tmp["id"] = ann_id
        ann_id += 1
        annotations_tmp["image_id"] = img_id
        annotations_tmp["segmentation"] = []
        annotations_tmp["ignore"] = 0
        annotations_tmp["area"] = (x2-x1)*(y2-y1)
        annotations_tmp["iscrowd"] = 0
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1]
        annotations_tmp["category_id"] = cats.index(label)

        coco_data["annotations"].append(annotations_tmp)
        j += 2


100%|████████████████████████████████████████| 320/320 [00:08<00:00, 39.26it/s]


In [15]:
outfile =  open(output_annotation_file, 'w')
json_str = json.dumps(coco_data, indent=4)
outfile.write(json_str)
outfile.close()